## Movie Recommendation
### 프로젝트 - Movielens 영화 추천 실습
***
> **1) 데이터 준비와 전처리
> Movielens 데이터는 rating.dat 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 깔끔하게 정리되어 있습니다.**  
> 
> **2) 분석해 봅시다.  
> -ratings에 있는 유니크한 영화 개수  
> -rating에 있는 유니크한 사용자 수  
> -가장 인기 있는 영화 30개(인기순)**  
> 
> **3) 내가 선호하는 영화를 5가지 골라서 rating에 추가해 줍시다.**  
> **4) CSR matrix를 직접 만들어 봅시다.**  
> **5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.**  
> **6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.**  
> **7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.**  
> **8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.**
***
**평가문항**  
> ***1. CSR matrix가 정상적으로 만들어졌다.  
> -사용자와 아이템 개수를 바탕으로 정확한 사이즈로 만들었다.***  
>  
> ***2. MF 모델이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다.   
> -사용자와 아이템 벡터 내적수치가 의미있게 형성되었다.***  
>  
> ***3. 비슷한 영화 찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다.  
> -MF모델이 예측한 유저 선호도 및 아이템간 유사도, 기여도가 의미있게 측정되었다.***  
***
**1) 데이터 준비와 전처리**

In [1]:
import pandas as pd
import os
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares
import numpy as np

rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 3점 이상만 남기기
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# ratings에서 timestamp 컬럼 삭제
ratings.drop(['timestamp'], axis=1, inplace=True)

In [14]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)
ratings['count']
ratings.head()

,user_id,movie_id,count
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


In [5]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


**2) 분석해 봅시다.**
**-ratings에 있는 유니크한 영화 개수
-rating에 있는 유니크한 사용자 수
-가장 인기 있는 영화 30개(인기순)**

In [6]:
# ratings에 있는 유니크한 영화 개수
ratings['movie_id'].nunique()

3628

In [7]:
# ratings에 있는 유니크한 사용자수
ratings['user_id'].nunique()

6039

In [8]:
# 데이터에 영화이름을 넣어도 되지않을까?
# 영화이름 넣었을때, 학습시 문제는?
# 포켓몬스터 참고하기

In [9]:
# ratings에 가장 인기많은 영화
movie_count = ratings.groupby('movie_id')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
Name: user_id, dtype: int64

In [10]:
# # 내가 좋아하는 영화 5개 고르기
# my_favorite = ['Home Alone (1990)' , 'Snow White and the Seven Dwarfs (1937)' ,'Aladdin (1992)' ,'Pinocchio (1940)' ,'Beauty and the Beast (1991)']

# # 'zimin'이라는 user_id가 위 아티스트의 노래를 30회씩 들었다고 가정하겠습니다.
# my_playlist = pd.DataFrame({'user_id': ['ethan']*5, 'movie_id': my_favorite, 'count':[5]*5})

# if not ratings.isin({'user_id':['ethan']})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
#     ratings = ratings.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

# ratings.tail(10) 

In [11]:
# # 고유한 유저, 아티스트를 찾아내는 코드
# user_unique = ratings['user_id'].unique()
# movie_unique = ratings['movie_id'].unique()

# # 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
# user_to_idx = {v:k for k,v in enumerate(user_unique)}
# movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [12]:
# print(user_to_idx['ethan'])    # 358869명의 유저 중 마지막으로 추가된 유저이니 358868이 나와야 합니다. 
# print(movie_to_idx['Home Alone (1990)'])